In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
import os
import json
from pandas.io.json import json_normalize
import pprint
# import ijson
import matplotlib.pyplot as plt
import string
import fnmatch
import glob

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
for dirname, _, filenames in os.walk('C:/Users/trivikram.cheedella/OneDrive - JD Power/Data Science Data/CORD-19-research-challenge'):
    for filename in filenames:
        # print(os.path.join(dirname, filename))
        pass

# for filename in fnmatch.filter(filenames, '*.txt'):
# Any results you write to the current directory are saved as output.

In [2]:
input = Path('C:/Users/trivikram.cheedella/OneDrive - JD Power/Data Science Data/CORD-19-research-challenge')
output = Path('C:/Users/trivikram.cheedella/OneDrive - JD Power/Data Science Data/CORD-19-research-challenge/output')

In [3]:
df_all_sources_metadata = pd.read_csv(input / 'metadata.csv')

In [4]:
print(df_all_sources_metadata.shape)
df_all_sources_metadata.info()

(44220, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44220 entries, 0 to 44219
Data columns (total 15 columns):
sha                            28462 non-null object
source_x                       44220 non-null object
title                          43996 non-null object
doi                            40750 non-null object
pmcid                          23319 non-null object
pubmed_id                      22943 non-null float64
license                        44220 non-null object
abstract                       35806 non-null object
publish_time                   34197 non-null object
authors                        41074 non-null object
journal                        33173 non-null object
Microsoft Academic Paper ID    964 non-null float64
WHO #Covidence                 1767 non-null object
has_full_text                  44220 non-null bool
full_text_file                 32829 non-null object
dtypes: bool(1), float64(2), object(12)
memory usage: 4.8+ MB


In [5]:
df_all_sources_metadata.head()

,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file
0,NaN,Elsevier,Intrauterine virus infections and congenital h...,10.1016/0002-8703(72)90077-4,NaN,4361535.0,els-covid,Abstract The etiologic basis for the vast majo...,1972-12-31,"Overall, James C.",American Heart Journal,NaN,NaN,False,custom_license
1,NaN,Elsevier,Coronaviruses in Balkan nephritis,10.1016/0002-8703(80)90355-5,NaN,6243850.0,els-covid,NaN,1980-03-31,"Georgescu, Leonida; Diosi, Peter; Buţiu, Ioan;...",American Heart Journal,NaN,NaN,False,custom_license
2,NaN,Elsevier,Cigarette smoking and coronary heart disease: ...,10.1016/0002-8703(80)90356-7,NaN,7355701.0,els-covid,NaN,1980-03-31,"Friedman, Gary D",American Heart Journal,NaN,NaN,False,custom_license
3,aecbc613ebdab36753235197ffb4f35734b5ca63,Elsevier,Clinical and immunologic studies in identical ...,10.1016/0002-9343(73)90176-9,NaN,4579077.0,els-covid,"Abstract Middle-aged female identical twins, o...",1973-08-31,"Brunner, Carolyn M.; Horwitz, David A.; Shann,...",The American Journal of Medicine,NaN,NaN,True,custom_license
4,NaN,Elsevier,Epidemiology of community-acquired respiratory...,10.1016/0002-9343(85)90361-4,NaN,4014285.0,els-covid,Abstract Upper respiratory tract infections ar...,1985-06-28,"Garibaldi, Richard A.",The American Journal of Medicine,NaN,NaN,False,custom_license


The following code for reading in the JSON file is taken from ...

https://www.kaggle.com/ivanegapratama/covid-eda-initial-exploration-tool

In [6]:
%%time
all_json = glob.glob(f'{input}/**/*.json', recursive=True)
len(all_json)

Wall time: 933 ms


29315

In [7]:
class FileReader:
    def __init__(self, file_path):
        with open(file_path) as file:
            content = json.load(file)
            self.paper_id = content['paper_id']
            self.abstract = []
            self.body_text = []
            # Abstract
            for entry in content['abstract']:
                self.abstract.append(entry['text'])
            # Body text
            for entry in content['body_text']:
                self.body_text.append(entry['text'])
            self.abstract = '\n'.join(self.abstract)
            self.body_text = '\n'.join(self.body_text)
            # Extend Here
            #
            #
    def __repr__(self):
        return f'{self.paper_id}: {self.abstract[:200]}... {self.body_text[:200]}...'
first_row = FileReader(all_json[0])
print(first_row)

0015023cc06b5362d332b3baf348d11567ca2fbb: word count: 194 22 Text word count: 5168 23 24 25 author/funder. All rights reserved. No reuse allowed without permission. Abstract 27 The positive stranded RNA genomes of picornaviruses comprise a si... VP3, and VP0 (which is further processed to VP2 and VP4 during virus assembly) (6). The P2 64 and P3 regions encode the non-structural proteins 2B and 2C and 3A, 3B (1-3) (VPg), 3C pro and 4 structura...


In [8]:
%%time
dict_ = {'paper_id': [], 'abstract': [], 'body_text': []}
for idx, entry in enumerate(all_json):
    if idx % (len(all_json) // 10) == 0:
        print(f'Processing index: {idx} of {len(all_json)}')
    content = FileReader(entry)
    dict_['paper_id'].append(content.paper_id)
    dict_['abstract'].append(content.abstract)
    dict_['body_text'].append(content.body_text)
df_covid = pd.DataFrame(dict_, columns=['paper_id', 'abstract', 'body_text'])
df_covid.head()

Processing index: 0 of 29315
Processing index: 2931 of 29315
Processing index: 5862 of 29315
Processing index: 8793 of 29315
Processing index: 11724 of 29315
Processing index: 14655 of 29315
Processing index: 17586 of 29315
Processing index: 20517 of 29315
Processing index: 23448 of 29315
Processing index: 26379 of 29315
Processing index: 29310 of 29315
Wall time: 2min 49s


,paper_id,abstract,body_text
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP..."
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by..."
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p..."


In [9]:
dict_ = None

In [10]:
%%time
df_covid['abstract_word_count'] = df_covid['abstract'].apply(lambda x: len(x.strip().split()))
df_covid['body_word_count'] = df_covid['body_text'].apply(lambda x: len(x.strip().split()))
df_covid.head()

Wall time: 24.5 s


,paper_id,abstract,body_text,abstract_word_count,body_word_count
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 Text word count: 5168 23 24...,"VP3, and VP0 (which is further processed to VP...",241,1728
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,The 2019-nCoV epidemic has spread across China...,0,755
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,Infectious bronchitis (IB) causes significant ...,"Infectious bronchitis (IB), which is caused by...",1647,4003
3,0139ea4ca580af99b602c6435368e7fdbefacb03,Nipah Virus (NiV) came into limelight recently...,Nipah is an infectious negative-sense single-s...,326,2399
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,Background: A novel coronavirus (2019-nCoV) em...,"In December 2019, a cluster of patients with p...",22,4642


In [11]:
%%time
df_covid['abstract'] = df_covid['abstract'].apply(lambda x: x.lower())
df_covid['body_text'] = df_covid['body_text'].apply(lambda x: x.lower())
df_covid.head()

Wall time: 17.1 s


,paper_id,abstract,body_text,abstract_word_count,body_word_count
0,0015023cc06b5362d332b3baf348d11567ca2fbb,word count: 194 22 text word count: 5168 23 24...,"vp3, and vp0 (which is further processed to vp...",241,1728
1,004f0f8bb66cf446678dc13cf2701feec4f36d76,,the 2019-ncov epidemic has spread across china...,0,755
2,00d16927588fb04d4be0e6b269fc02f0d3c2aa7b,infectious bronchitis (ib) causes significant ...,"infectious bronchitis (ib), which is caused by...",1647,4003
3,0139ea4ca580af99b602c6435368e7fdbefacb03,nipah virus (niv) came into limelight recently...,nipah is an infectious negative-sense single-s...,326,2399
4,013d9d1cba8a54d5d3718c229b812d7cf91b6c89,background: a novel coronavirus (2019-ncov) em...,"in december 2019, a cluster of patients with p...",22,4642


In [12]:
df_covid.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
paper_id,29315,29315,ac81102667b0d56edeb8ab0044765dc49a19f374,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract,29315,21049,,8051,NaN,NaN,NaN,NaN,NaN,NaN,NaN
body_text,29315,29117,"in previous reports, workers have characterize...",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract_word_count,29315,NaN,NaN,NaN,164.501,175.353,0,0,161,240,4767
body_word_count,29315,NaN,NaN,NaN,4802.29,7322.71,1,2444,3730,5509.5,260378


In [13]:
df_covid.drop_duplicates(['body_text'], inplace=True)
df_covid.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
paper_id,29117,29117,ac81102667b0d56edeb8ab0044765dc49a19f374,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract,29117,21044,,8004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
body_text,29117,29117,multiple sclerosis (ms) is an inflammatory dem...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
abstract_word_count,29117,NaN,NaN,NaN,164.504,175.637,0,0,160,240,4767
body_word_count,29117,NaN,NaN,NaN,4811.94,7344.37,1,2450,3733,5513,260378


In [14]:
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS

In [15]:
lemmatizer = WordNetLemmatizer()

print("Number of stopwrods from STOPWORDS: ", len(STOPWORDS))
print("Number of stopwrods from stopwords.words('english'): ", len(stopwords.words('english')))
other_stopwords = ['q', 'license', 'preprint', 'copyright', 'http', 'doi', 'preprint', 'copyright', 'peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 'rights', 'reserved', 'permission', 'used', 'using', 'biorxiv', 'fig', 'fig.', 'al.', 'di', 'la', 'il', 'del', 'le', 'della', 'dei', 'delle', 'una', 'da',  'dell',  'non', 'si']
custom_stopwords = []
custom_stopwords = list(set(stopwords.words('english') + list(STOPWORDS))) + other_stopwords

print("Number of stopwrods from custom_stopwords: ", len(custom_stopwords))
print(custom_stopwords)

Number of stopwrods from STOPWORDS:  190
Number of stopwrods from stopwords.words('english'):  179
Number of stopwrods from custom_stopwords:  263
['so', 'they', 'just', "she's", "i'd", 'she', 'than', 'too', 'doing', "mightn't", "we'll", 'theirs', 'have', 'do', "how's", 'were', "doesn't", 'all', 'both', 'into', 'her', "didn't", "they'd", 'over', 'nor', 'been', 'yourself', "you'll", 'about', 'how', 'other', "can't", 'from', 'could', 'however', 'until', 'through', 'before', 'in', 'o', "where's", "don't", 'against', 'isn', 'hers', "won't", 'would', "you'd", 'shouldn', "shan't", 'aren', 'com', 'his', 'am', 'had', 'shan', 'an', "you're", 'such', 'it', 'which', 'be', 've', 'like', "isn't", 'further', "you've", 'didn', 'its', 'while', "we'd", 'this', 'www', 'their', 'is', 'above', 'then', 's', "that'll", "mustn't", 'there', 'any', "couldn't", 'now', 'after', 'me', 'very', 'why', 'we', "aren't", 'haven', 'ain', "should've", 'that', "he's", 'our', 'also', 'yourselves', 'else', 'ever', 'otherwis

In [16]:
def remove_punc_stopword_tokens(tokens, tolower=False):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    remove_punc = [word for word in tokens if word not in string.punctuation]
    if tolower:
        return [word.lower() for word in remove_punc if word.lower() not in custom_stopwords]
    else:
        return [word for word in remove_punc if word not in custom_stopwords]

In [17]:
def remove_punc_stopword_text(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    remove_punc = [word for word in text if word not in string.punctuation]
    remove_punc = ''.join(remove_punc)
    return [word.lower() for word in remove_punc.split() if word.lower() not in custom_stopwords]

In [18]:
def clean_the_text(text):
        tokens = word_tokenize(text)
        remove_punc = [word for word in tokens if word not in string.punctuation]
        remove_stopwords = [word.lower() for word in remove_punc if word.lower() not in custom_stopwords]
        more_than_three = [w for w in remove_stopwords if len(w)>3]
        lem = [lemmatizer.lemmatize(w) for w in more_than_three]
        return ' '.join(lem)

# cleaned_text = clean_the_text("my name is TRIVIKRAM, and I am first born in my fam don't process processing patients")
# print(cleaned_text)

In [19]:
df_covid_for_nlp = df_covid.copy()
df_covid = None

In [20]:
%%time
df_covid_for_nlp['cleaned_text'] = df_covid_for_nlp['body_text'].apply(lambda x: clean_the_text(x))
# print(cleaned_text[:20])

Wall time: 58min 49s


In [21]:
df_covid_for_nlp['cleaned_text'].head()

0    processed virus assembly region encode non-str...
1    2019-ncov epidemic spread across china country...
2    infectious bronchitis caused infectious bronch...
3    nipah infectious negative-sense single-strande...
4    december 2019 cluster patient pneumonia unknow...
Name: cleaned_text, dtype: object

In [22]:
df_covid_for_nlp_10K = df_covid_for_nlp.head(10000)
df_covid_for_nlp_10K.shape

(10000, 6)

# Topic Modeling using Latent Dirichlet Allocation (LDA)

In [23]:
%%time

from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = count_vect.fit_transform(df_covid_for_nlp_10K['cleaned_text'].values.astype('U'))

Wall time: 5min 41s


In [24]:
doc_term_matrix

<10000x110226 sparse matrix of type '<class 'numpy.int64'>'
	with 7517951 stored elements in Compressed Sparse Row format>

In [25]:
%%time

from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=50, random_state=42, n_jobs=-1)
LDA.fit(doc_term_matrix)

Wall time: 9min 20s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=50, n_jobs=-1,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [26]:
import random

for i in range(10):
    random_id = random.randint(0,len(count_vect.get_feature_names()))
    print(count_vect.get_feature_names()[random_id])

peif2a
islk
collections
pbmsf7c
fab
glioma
bioinformation
machine
toning
ck04


In [27]:
first_topic = LDA.components_[0]

In [28]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([106325,  96144,  76161,  86412, 105990,  53255, 106475,  23334,
        63943,  82451], dtype=int64)

In [29]:
for i in top_topic_words:
    print(count_vect.get_feature_names()[i])

viral
structure
pcv2
replication
vesicle
infected
virus
cell
membrane
prrsv


In [30]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['viral', 'structure', 'pcv2', 'replication', 'vesicle', 'infected', 'virus', 'cell', 'membrane', 'prrsv']


Top 10 words for topic #1:
['stem', 'ribosome', 'frame', 'trna', 'frameshifting', 'sequence', 'translation', 'site', 'mrna', 'codon']


Top 10 words for topic #2:
['cell', 'genome', 'interaction', 'ifitm3', 'host', 'replication', 'domain', 'viral', 'virus', 'protein']


Top 10 words for topic #3:
['method', 'positive', 'probe', 'reaction', 'pcr', 'rt', 'detection', 'primer', 'assay', 'sample']


Top 10 words for topic #4:
['host', 'immune', 'activation', 'ifn', 'type', 'protein', 'response', 'viral', 'virus', 'cell']


Top 10 words for topic #5:
['protein', 'vector', 'human', 'immune', 'virus', 'antigen', 'antibody', 'response', 'cell', 'vaccine']


Top 10 words for topic #6:
['anti', 'immune', 'level', 'macrophage', 'response', 'cytokine', 'expression', 'mouse', 'il', 'cell']


Top 10 words for topic #7:
['coronaviruses', 'hcov', 'coronavirus', 'camel

In [31]:
topic_values = LDA.transform(doc_term_matrix)
topic_values.shape

(10000, 50)

# Non-Negative Matrix Factorization (NMF)

In [32]:
%%time 

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words='english')
doc_term_matrix = tfidf_vect.fit_transform(df_covid_for_nlp_10K['cleaned_text'].values.astype('U'))

Wall time: 5min 32s


In [33]:
doc_term_matrix

<10000x110226 sparse matrix of type '<class 'numpy.float64'>'
	with 7517951 stored elements in Compressed Sparse Row format>

In [34]:
%%time 

from sklearn.decomposition import NMF

nmf = NMF(n_components=50, random_state=42)
nmf.fit(doc_term_matrix )

Wall time: 3min 45s


NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=50, random_state=42, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [35]:
import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

pomp
survey
pparg
m19921
26k
verdinexor
annelid
ethoxy
craniata
drago


In [36]:
first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]

In [37]:
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

lncrna
3p
sirna
mrna
5p
lncrnas
ev71
mirna
mirnas
mir


In [38]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['lncrna', '3p', 'sirna', 'mrna', '5p', 'lncrnas', 'ev71', 'mirna', 'mirnas', 'mir']


Top 10 words for topic #1:
['diagnosis', 'care', 'severe', 'respiratory', 'treatment', 'clinical', 'pneumonia', 'admission', 'hospital', 'patient']


Top 10 words for topic #2:
['10', 'reuse', 'author', 'org', 'doi', 'funder', '1101', 'peer', 'holder', 'reviewed']


Top 10 words for topic #3:
['amino', 'isolates', 'recombination', 'genotype', 'nucleotide', 'phylogenetic', 'tree', 'genome', 'strain', 'sequence']


Top 10 words for topic #4:
['rbd', 'east', 'korea', 'hdpp4', 'human', 'arabia', 'saudi', 'dpp4', 'cov', 'mers']


Top 10 words for topic #5:
['probability', 'simulation', 'value', 'individual', 'estimate', 'rate', 'population', 'parameter', 'epidemic', 'model']


Top 10 words for topic #6:
['orf3', 'vero', 'feed', 'diarrhea', 'farm', 'cv777', 'strain', 'pig', 'piglet', 'pedv']


Top 10 words for topic #7:
['particle', 'genome', 'antiviral', 'infected', 'respiratory

In [ ]:
topic_values = nmf.transform(doc_term_matrix)
# reviews_datasets['Topic'] = topic_values.argmax(axis=1)
# reviews_datasets.head()